### Fetching The 7z archive

In [0]:
try:
    dbutils.fs.ls("file:/databricks/driver/dblp.v13.7z")
    print("Archive in filesystem (file:/databricks/driver/dblp.v13.7z)")
except:
    try:
        dbutils.fs.ls("dbfs:/FileStore/data/dblp.v13.7z")
        print("Archive located in FileStore. Copying into local store..")
        dbutils.fs.cp("dbfs:/FileStore/data/dblp.v13.7z", "file:/databricks/driver/dblp.v13.7z")
        print("Completed")
    except:
        print("7z archive not found. Fetching from URL...")
        !wget https://originalstatic.aminer.cn/misc/dblp.v13.7z
        print("7z archive Downloaded. Moving archive to FileStore..")
        dbutils.fs.mkdirs("dbfs:/FileStore/data")
        dbutils.fs.cp("file:/databricks/driver/dblp.v13.7z", "dbfs:/FileStore/data/dblp.v13.7z")
        print("Completed.")

7z archive not found. Fetching from URL...
--2022-04-15 20:01:54--  https://originalstatic.aminer.cn/misc/dblp.v13.7z
Resolving originalstatic.aminer.cn (originalstatic.aminer.cn)... 159.27.2.14
Connecting to originalstatic.aminer.cn (originalstatic.aminer.cn)|159.27.2.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2568255035 (2.4G) [application/x-7z-compressed]
Saving to: ‘dblp.v13.7z’

dblp.v13.7z

*** WARNING: max output size exceeded, skipping output. ***

dblp.v13.7z         100%[===================>]   2.39G  9.17MB/s    in 4m 50s  

2022-04-15 20:06:44 (8.45 MB/s) - ‘dblp.v13.7z’ saved [2568255035/2568255035]

7z archive Downloaded. Moving archive to FileStore..
Completed.


In [0]:
dbutils.fs.ls("file:/databricks/driver/dblp.v13.7z")

Out[42]: [FileInfo(path='file:/databricks/driver/dblp.v13.7z', name='dblp.v13.7z', size=2568255035, modificationTime=1631501339000)]

### Extracting Archive into json

In [0]:
!pip install py7zr -q

You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.


In [0]:
import py7zr

archive = py7zr.SevenZipFile('dblp.v13.7z', mode='r')
archive.extractall()
archive.close()

In [0]:
dbutils.fs.ls("file:/databricks/driver/dblpv13.json")

Out[45]: [FileInfo(path='file:/databricks/driver/dblpv13.json', name='dblpv13.json', size=17352640799, modificationTime=1631240655155)]

### Cleaning NumberInt(#) tags

In [0]:
import re

# Cleaning the `NumberInt` tag
fin = open(f"dblpv13.json")
fout = open(f"dblpv13_clean.json", "wt")
for line in fin:
    fout.write(re.sub(r"NumberInt\([\d]*\)", lambda x: "".join(re.findall(r"\d", x.group(0))), line))
fin.close()
fout.close()

### Partitioning Dataset into JSON files

In [0]:
!pip install ijson tqdm -q

You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.


In [0]:
%mkdir data

In [0]:
import ijson
import json
from tqdm import tqdm
import decimal

class DecimalEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, decimal.Decimal):
            return str(o)
        return super(DecimalEncoder, self).default(o)

data_dir = 'data/'
with open('dblpv13_clean.json', 'r') as f:
    counter, file_id = 0, 0
    file_buffer = []
    for obj_data in tqdm(ijson.items(f, 'item')):
        file_buffer.append(obj_data)
        counter += 1
        if counter % 300000 == 0:
            print(f" Saving, data_PART_{file_id}.json in {data_dir}")
            f = open(f'{data_dir}data_PART_{file_id}.json', 'w')
            dump = json.dumps(file_buffer, cls=DecimalEncoder)
            f.write(dump)
            f.close()
            file_id += 1
            file_buffer = []
f = open(f'{data_dir}data_PART_{file_id}.json', 'w')
dump = json.dumps(file_buffer, cls=DecimalEncoder)
print(f" Saving, data_PART_{file_id}.json in {data_dir}")
f.write(dump)
f.close()
file_id += 1
file_buffer = []

298680it [00:19, 18782.17it/s] Saving, data_PART_0.json in data/
598810it [00:50, 19435.19it/s] Saving, data_PART_1.json in data/
899960it [01:23, 19491.69it/s] Saving, data_PART_2.json in data/
1198874it [01:55, 18848.56it/s] Saving, data_PART_3.json in data/
1498007it [02:27, 19529.21it/s] Saving, data_PART_4.json in data/
1645073it

*** WARNING: max output size exceeded, skipping output. ***

3899024it [06:44, 19311.96it/s] Saving, data_PART_12.json in data/
4199119it [07:17, 16566.68it/s] Saving, data_PART_13.json in data/
4498341it [07:53, 15757.19it/s] Saving, data_PART_14.json in data/
4799983it [08:27, 18444.90it/s] Saving, data_PART_15.json in data/
5099137it [09:04, 12606.14it/s] Saving, data_PART_16.json in data/
5354309it [09:32, 9354.16it/s] 
 Saving, data_PART_17.json in data/


In [0]:
len(file_buffer)

Out[50]: 0

In [0]:
path = "file:/databricks/driver/data/"
[f"{path}data_PART_{num}.json" for num in range(file_id)]

Out[55]: ['file:/databricks/driver/data/data_PART_0.json',
 'file:/databricks/driver/data/data_PART_1.json',
 'file:/databricks/driver/data/data_PART_2.json',
 'file:/databricks/driver/data/data_PART_3.json',
 'file:/databricks/driver/data/data_PART_4.json',
 'file:/databricks/driver/data/data_PART_5.json',
 'file:/databricks/driver/data/data_PART_6.json',
 'file:/databricks/driver/data/data_PART_7.json',
 'file:/databricks/driver/data/data_PART_8.json',
 'file:/databricks/driver/data/data_PART_9.json',
 'file:/databricks/driver/data/data_PART_10.json',
 'file:/databricks/driver/data/data_PART_11.json',
 'file:/databricks/driver/data/data_PART_12.json',
 'file:/databricks/driver/data/data_PART_13.json',
 'file:/databricks/driver/data/data_PART_14.json',
 'file:/databricks/driver/data/data_PART_15.json',
 'file:/databricks/driver/data/data_PART_16.json',
 'file:/databricks/driver/data/data_PART_17.json']

In [0]:
# https://stackoverflow.com/a/42540517 (This seems more proper way to merge the frames..)

# from functools import reduce

# path = "file:/databricks/driver/data/"
# dataframes = map(lambda r: spark.read.json(r), [f"{path}data_PART_{num}.json" for num in range(file_id)])
# union = reduce(lambda df1, df2: df1.unionAll(df2), dataframes)

dataframes = []
for num in range(file_id):
    print(f"Reading file data_PART_{num}.json into spark")
    dataframes.append(spark.read.json(f"{path}data_PART_{num}.json"))
print(len(dataframes))

Reading file data_PART_0.json into spark
Reading file data_PART_1.json into spark
Reading file data_PART_2.json into spark
Reading file data_PART_3.json into spark
Reading file data_PART_4.json into spark
Reading file data_PART_5.json into spark
Reading file data_PART_6.json into spark
Reading file data_PART_7.json into spark
Reading file data_PART_8.json into spark
Reading file data_PART_9.json into spark
Reading file data_PART_10.json into spark
Reading file data_PART_11.json into spark
Reading file data_PART_12.json into spark
Reading file data_PART_13.json into spark
Reading file data_PART_14.json into spark
Reading file data_PART_15.json into spark
Reading file data_PART_16.json into spark
Reading file data_PART_17.json into spark
18


In [0]:
dataframes

Out[67]: [DataFrame[_id: string, abstract: string, authors: array<struct<_id:string,bio:string,email:string,gid:string,name:string,name_zh:string,oid:string,oid_zh:string,orcid:string,org:string,org_zh:string,orgid:string,orgs:array<string>,orgs_zh:array<string>,sid:string>>, doi: string, fos: array<string>, isbn: string, issn: string, issue: string, keywords: array<string>, lang: string, n_citation: bigint, page_end: string, page_start: string, pdf: string, references: array<string>, title: string, url: array<string>, venue: struct<_id:string,issn:string,name:string,name_d:string,name_s:string,online_issn:string,publisher:string,raw:string,raw_zh:string,sid:string,src:string,t:string,type:bigint>, volume: string, year: bigint],
 DataFrame[_id: string, abstract: string, authors: array<struct<_id:string,bio:string,email:string,gid:string,name:string,name_zh:string,oid:string,oid_zh:string,orcid:string,org:string,org_zh:string,orgid:string,orgs:array<string>,orgs_zh:array<string>,sid:str

In [0]:
dataframes[0].printSchema()

root
 |-- _id: string (nullable = true)
 |-- abstract: string (nullable = true)
 |-- authors: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- _id: string (nullable = true)
 |    |    |-- bio: string (nullable = true)
 |    |    |-- email: string (nullable = true)
 |    |    |-- gid: string (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- name_zh: string (nullable = true)
 |    |    |-- oid: string (nullable = true)
 |    |    |-- oid_zh: string (nullable = true)
 |    |    |-- orcid: string (nullable = true)
 |    |    |-- org: string (nullable = true)
 |    |    |-- org_zh: string (nullable = true)
 |    |    |-- orgid: string (nullable = true)
 |    |    |-- orgs: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- orgs_zh: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- sid: string (nullable = true)
 |-- doi: string (nulla

In [0]:
dataframes[8].printSchema()

root
 |-- _id: string (nullable = true)
 |-- abstract: string (nullable = true)
 |-- authors: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- _id: string (nullable = true)
 |    |    |-- bio: string (nullable = true)
 |    |    |-- email: string (nullable = true)
 |    |    |-- gid: string (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- name_zh: string (nullable = true)
 |    |    |-- oid: string (nullable = true)
 |    |    |-- oid_zh: string (nullable = true)
 |    |    |-- orcid: string (nullable = true)
 |    |    |-- org: string (nullable = true)
 |    |    |-- org_zh: string (nullable = true)
 |    |    |-- orgid: string (nullable = true)
 |    |    |-- orgs: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- orgs_zh: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- position: string (nullable = true)
 |    |    |-- sid

In [0]:
# union.printSchema()

Out[59]: <map at 0x7fb678157ca0>

In [0]:
mapFrames = map(lambda r: r, dataframes)
union = reduce(lambda df1, df2: df1.unionAll(df2), mapFrames)


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
<command-2656773693406771> in <module>
      1 mapFrames = map(lambda r: r, dataframes)
----> 2 union = reduce(lambda df1, df2: df1.unionAll(df2), mapFrames)

<command-2656773693406771> in <lambda>(df1, df2)
      1 mapFrames = map(lambda r: r, dataframes)
----> 2 union = reduce(lambda df1, df2: df1.unionAll(df2), mapFrames)

/databricks/spark/python/pyspark/sql/dataframe.py in unionAll(self, other)
   1995         Also as standard in SQL, this function resolves columns by position (not by name).
   1996         """
-> 1997         return self.union(other)
   1998 
   1999     def unionByName(self, other, allowMissingColumns=False):

/databricks/spark/python/pyspark/sql/dataframe.py in union(self, other)
   1983         Also as standard in SQL, this function resolves columns by position (not by name).
   1984         """
-> 1985        

In [0]:
# dbutils.fs.mkdirs("dbfs:/FileStore/data/split_data")

# dbutils.fs.cp("file:/databricks/driver/data/", "dbfs:/FileStore/data/split_data", recurse = True)